# WARNING - This file can take 10+ minutes to execute

In [1]:
#initial inspection of the resulting data
import pandas as pd
import numpy as np
#open the csv file as a data frame
df = pd.read_csv('UFO_observations_1994_2018_2.csv', index_col=0)
pd.set_option('max_rows', 10)

In [2]:
#converts the date strings into date/time formatted objects
df["Date and Time"] = pd.to_datetime(df["Date and Time"])
df["Date Posted"] = pd.to_datetime(df["Date Posted"])

#scripts to clean up the dates as anything before 1970 is being automatically assigned to the future.
#i.e.: 1/1/46 is turning into 1/1/2047 - see source below 
#https://stackoverflow.com/questions/37766353/pandas-to-datetime-parsing-wrong-year

from datetime import datetime, timedelta, date
future = df["Date and Time"] > date(year=2019,month=1,day=1)
df.loc[future, "Date and Time"] -= timedelta(days=365*100)

#tidy up of any NaN values in any cells
for col in df.columns[1:5]:
    df[col] = df[col].replace(np.nan, "")


C:\Users\chris\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
#calculates the overall difference between the date posted on the site and the sighting itself
#this is at the mercy of the site admins and may not reflect the actual delay between the sighting and the report
#as such this field may have limited value for reporting

df['Report_Lapse'] = (df['Date Posted']-df['Date and Time']).dt.days
df.head()

,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"Fireball descending towards ground, changes tr...",2014-11-06,7589
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and Isaw saucer craft -silent -eight...,2000-07-11,2359
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,Large Silver Disk explodes over SD Antenna Fie...,2002-08-28,3142
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,The objects hover to the west changing diffren...,2004-01-22,3655
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,Fire ball after northidge quake .,2013-12-23,7279


In [4]:
#calculates the hour of the sighting 
df['Time'] = df['Date and Time'].dt.hour

df.head()

,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse,Time
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"Fireball descending towards ground, changes tr...",2014-11-06,7589,17
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and Isaw saucer craft -silent -eight...,2000-07-11,2359,22
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,Large Silver Disk explodes over SD Antenna Fie...,2002-08-28,3142,19
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,The objects hover to the west changing diffren...,2004-01-22,3655,19
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,Fire ball after northidge quake .,2013-12-23,7279,5


In [5]:
#progress bar function for use by other functions
def progress(row, lastrow):
    increment = int(lastrow * 0.01)
    endrow = increment * 50
    if row == 1:
        print("Working through", lastrow, "data entries. This might take a while.")
        print("Progress", end="")
    if (row%increment)== 0:
        print(".", end="")
    if (row%endrow)== 0:
        print(".")
    if (row) == lastrow:
        print("")
        print("Complete")
  

In [6]:
# function to derive the number of witnesses to the sighting based on content in the Summary field
# default assumption is a single witness
# the Summary field strings are split in the comarison to avoid false positives i.e.: 'we' in 'were'
df['Summary'] = df['Summary'].astype(str).str.lower()
df['Witnesses'] = "Solo"
group = ("us", "we")
count = 1

def extractwitnesses (text):
    sum = 0
    for x in group:
        sum = sum + text.split().count(x)
    if sum > 0: 
        return("Group")
    else: return("Solo")

for row in df.index:
    progress(count, len(df.index))
    text = df.loc[row, 'Summary']
    df.loc[row,'Witnesses'] = extractwitnesses(text)
    count = count + 1
    
df.head()

Working through 102725 data entries. This might take a while.
Progress...................................................
...................................................

Complete


,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse,Time,Witnesses
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"fireball descending towards ground, changes tr...",2014-11-06,7589,17,Solo
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and isaw saucer craft -silent -eight...,2000-07-11,2359,22,Solo
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,large silver disk explodes over sd antenna fie...,2002-08-28,3142,19,Solo
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,the objects hover to the west changing diffren...,2004-01-22,3655,19,Solo
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,fire ball after northidge quake .,2013-12-23,7279,5,Solo


In [7]:
#function to create a field for the day of the UFO sighting as named day i.e.: "Thursday"
count = 1
for row in df.index:
    progress(count, len(df.index))
    sighting = df.loc[row,'Date and Time'] 
    df.loc[row, 'Sighting Day'] = sighting.strftime("%A")
    count = count + 1
df.head()

Working through 102725 data entries. This might take a while.
Progress...................................................
...................................................

Complete


,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse,Time,Witnesses,Sighting Day
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"fireball descending towards ground, changes tr...",2014-11-06,7589,17,Solo,Tuesday
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and isaw saucer craft -silent -eight...,2000-07-11,2359,22,Solo,Monday
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,large silver disk explodes over sd antenna fie...,2002-08-28,3142,19,Solo,Wednesday
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,the objects hover to the west changing diffren...,2004-01-22,3655,19,Solo,Tuesday
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,fire ball after northidge quake .,2013-12-23,7279,5,Solo,Monday


In [8]:
# function to extract any colours mentioned from the summary field
# note only the first found value is returned 

colours = ("blue", "black", "brown", "green", "orange", "red", "violet", "purple", "yellow", "silver")

def extractcolours (text):
    for x in colours:
        if x in text:
            return(x)
            break
    return("None")

count = 1
for row in df.index:
    progress(count, len(df.index))
    text = df.loc[row, 'Summary']
    df.loc[row,'Colour'] = extractcolours(text)
    count = count + 1
    
df.head()

Working through 102725 data entries. This might take a while.
Progress...................................................
...................................................

Complete


,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse,Time,Witnesses,Sighting Day,Colour
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"fireball descending towards ground, changes tr...",2014-11-06,7589,17,Solo,Tuesday,None
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and isaw saucer craft -silent -eight...,2000-07-11,2359,22,Solo,Monday,None
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,large silver disk explodes over sd antenna fie...,2002-08-28,3142,19,Solo,Wednesday,silver
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,the objects hover to the west changing diffren...,2004-01-22,3655,19,Solo,Tuesday,None
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,fire ball after northidge quake .,2013-12-23,7279,5,Solo,Monday,None


In [9]:
# function used to return the numeric values in the 'Duration' string field
# the default assumption is to return the largest value reported
# for example anything reported as lasting '5-7 minutes' would be considered as lasting 7 minutes
import re
def extractnumbers(text):
    templist = re.findall("\d+", text)
    if templist != []:
        x = max(templist)
    else:
        x = 0
    return(x)

count = 1

for row in df.index:
    progress(count, len(df.index))
    text = df.loc[row, 'Duration']
    df.loc[row,'Duration_Num'] = extractnumbers(text)
    count = count + 1

df['Duration_Num'] = df['Duration_Num'].astype(float)

df.head()

Working through 102725 data entries. This might take a while.
Progress...................................................
...................................................

Complete


,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse,Time,Witnesses,Sighting Day,Colour,Duration_Num
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"fireball descending towards ground, changes tr...",2014-11-06,7589,17,Solo,Tuesday,None,7.0
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and isaw saucer craft -silent -eight...,2000-07-11,2359,22,Solo,Monday,None,5.0
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,large silver disk explodes over sd antenna fie...,2002-08-28,3142,19,Solo,Wednesday,silver,10.0
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,the objects hover to the west changing diffren...,2004-01-22,3655,19,Solo,Tuesday,None,1.0
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,fire ball after northidge quake .,2013-12-23,7279,5,Solo,Monday,None,0.0


In [10]:
#function to calculate the total duration of a sighting in minutes

import re
df['Duration_Text'] = "hours"
df['Duration'] = df['Duration'].astype(str).str.lower()
df['Summary'] = df['Summary'].astype(str).str.lower()

def calculate_duration(row):
    if df.loc[row,'Duration_Text'] == "seconds":
        return(1/60) 
    if df.loc[row,'Duration_Text'] == "minutes":
        return(1) 
    if df.loc[row,'Duration_Text'] == "hours":
        return(60)

count = 1
for row in df.index:
    progress(count, len(df.index))
    text = df.loc[row, 'Duration']
    if re.search("sec", text):
        df.loc[row,'Duration_Text'] = "seconds"
    else:
        if re.search("min", text):
            df.loc[row, 'Duration_Text']="minutes"
    df.loc[row,'Total_Duration'] = calculate_duration(row)
    count = count + 1

df[['Total_Duration', 'Duration_Num']] = df[['Total_Duration', 'Duration_Num']].astype(float)
df['Total_Duration'] = df['Total_Duration']*df['Duration_Num']

df.head()

Working through 102725 data entries. This might take a while.
Progress...................................................
...................................................

Complete


,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse,Time,Witnesses,Sighting Day,Colour,Duration_Num,Duration_Text,Total_Duration
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"fireball descending towards ground, changes tr...",2014-11-06,7589,17,Solo,Tuesday,None,7.0,minutes,7.0
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and isaw saucer craft -silent -eight...,2000-07-11,2359,22,Solo,Monday,None,5.0,minutes,5.0
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,large silver disk explodes over sd antenna fie...,2002-08-28,3142,19,Solo,Wednesday,silver,10.0,minutes,10.0
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,the objects hover to the west changing diffren...,2004-01-22,3655,19,Solo,Tuesday,None,1.0,hours,60.0
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,fire ball after northidge quake .,2013-12-23,7279,5,Solo,Monday,None,0.0,seconds,0.0


In [11]:
df = df.drop(['Duration_Num', 'Duration_Text'], axis=1)
df.head()

,Date and Time,City,State,Shape,Duration,Summary,Date Posted,Report_Lapse,Time,Witnesses,Sighting Day,Colour,Total_Duration
1,1994-01-25 17:56:00,Spring Valley/LaMesa,California,Shapeless Light,5-7 minutes,"fireball descending towards ground, changes tr...",2014-11-06,7589,17,Solo,Tuesday,None,7.0
2,1994-01-24 22:00:00,Los Angeles,California,Disk,5 minutes,my friend and isaw saucer craft -silent -eight...,2000-07-11,2359,22,Solo,Monday,None,5.0
3,1994-01-19 19:00:00,Summit,South Dakota,Disk,10 minutes,large silver disk explodes over sd antenna fie...,2002-08-28,3142,19,Solo,Wednesday,silver,10.0
4,1994-01-18 19:00:00,Milford,Iowa,Shapeless Light,1 hour,the objects hover to the west changing diffren...,2004-01-22,3655,19,Solo,Tuesday,None,60.0
5,1994-01-17 05:00:00,Los Angeles,California,Shapeless Light,few seconds,fire ball after northidge quake .,2013-12-23,7279,5,Solo,Monday,None,0.0


In [12]:
df.to_csv('UFO_observations_1994_2018_3.csv', encoding='utf-8')